In [42]:
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt
faces = fetch_lfw_people(min_faces_per_person=60)

In [43]:
x,y = faces.data,faces.target
target_names = faces.target_names
n_samples,h,w = faces.images.shape
print('超过60张图片的名人:',target_names)
print('总的样本数量:',n_samples,h,w)

In [44]:
from sklearn.decomposition import PCA
pca = PCA(n_components=150,svd_solver='randomized',whiten=True,random_state=70).fit(x)
eigenfaces = pca.components_.reshape((150,h,w))
x_pca = pca.transform(x)
fig,ax = plt.subplots(3,5)
for i,axi in enumerate(ax.flat):
    axi.imshow(eigenfaces[i].reshape(h,w),cmap='bone')
    axi.set(xticks=[],yticks=[])
plt.show()

In [45]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

x_train,x_test,y_train,y_test = train_test_split(x_pca,y,test_size=400,random_state=42)
param_grid = {'C':[1,5,10,50,100],'gamma':[0.0001,0.0005,0.001,0.005,0.01,0.1]}
grid = GridSearchCV(SVC(kernel='rbf',random_state=0),param_grid,cv=5)
grid.fit(x_train,y_train)
print('最优参数值为:',grid.best_params_)

In [46]:
model = grid.best_estimator_
pred = model.predict(x_test)
re = classification_report(y_test,pred,target_names=faces.target_names)
print('最优模型的评估报告\n',re)

In [ ]:
fig ,ax = plt.subplots(3,5)
for i,axi in enumerate(ax.flat):
    axi.imshow(x_test[i].reshape(h,w),cmap='bone')
    axi.set(xticks=[],yticks=[])
    box = dict(fc='red',alpha=0.5)
    axi.set_ylabel(faces.target_names[pred[i]].split()[-1],bbox=None if pred[i] == y_test[i] else box)